# Les 7: Backpropagation

**Mathematical Foundations - IT & Artificial Intelligence**

---

## 7.0 Recap en Motivatie

In de vorige lessen hebben we alle puzzelstukjes verzameld:

- **Les 2-4**: Lineaire algebra - hoe data door een netwerk stroomt (forward pass)
- **Les 5**: Afgeleiden en de kettingregel - hoe we verandering meten
- **Les 6**: Gradient descent - hoe we parameters optimaliseren

Nu brengen we alles samen. We weten dat we de gradiënt van de loss naar elke parameter nodig hebben voor gradient descent. Maar in een netwerk met meerdere lagen, hoe bereken je de gradiënt naar een weight in de eerste laag?

Het antwoord is **backpropagation**: een algoritme dat de kettingregel efficiënt door het hele netwerk toepast. Het is de sleutel tot het trainen van diepe neurale netwerken.

Aan het einde van deze les zul je een neuraal netwerk from scratch bouwen en trainen op MNIST!

## 7.1 Leerdoelen

Na deze les begrijp je waarom backpropagation nodig is. Je kunt een neuraal netwerk voorstellen als een computational graph. Je begrijpt hoe de kettingregel door het netwerk wordt toegepast. Je kunt backpropagation met de hand uitvoeren voor kleine netwerken. Je kunt een volledig trainingsloop implementeren. Je kunt een neuraal netwerk trainen op MNIST.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, suppress=True)
np.random.seed(42)

print("Libraries geladen!")

## 7.2 De Computational Graph

### Het netwerk als een graaf

We kunnen een neuraal netwerk voorstellen als een gerichte graaf van operaties:

- **Nodes**: Operaties (matrixvermenigvuldiging, optelling, activatiefuncties, loss)
- **Edges**: Data die tussen operaties stroomt

De forward pass berekent de output door van input naar output te gaan. De backward pass berekent gradiënten door van output naar input te gaan.

### Voorbeeld

Beschouw een simpel netwerk: y = σ(Wx + b)

De computational graph is:
```
x → [×W] → z₁ → [+b] → z₂ → [σ] → y → [Loss] → L
```

In [ ]:
# Voorbeeld: forward pass door een simpel netwerk
def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

# Parameters
W = np.array([[0.5, -0.3],
              [0.2, 0.8]])
b = np.array([0.1, -0.2])

# Input
x = np.array([1.0, 2.0])

# Forward pass - stap voor stap
z1 = W @ x       # Matrixvermenigvuldiging
z2 = z1 + b      # Bias optellen
y = sigmoid(z2)  # Activatie

print("Forward Pass:")
print(f"Input x = {x}")
print(f"z₁ = W @ x = {z1}")
print(f"z₂ = z₁ + b = {z2}")
print(f"y = σ(z₂) = {y}")

## 7.3 Backpropagation: Het Idee

### Het probleem

We willen ∂L/∂W berekenen: hoe verandert de loss als we W veranderen? Het probleem is dat W niet direct verbonden is met L - er zitten meerdere operaties tussen.

### De oplossing: kettingregel

We passen de kettingregel toe, maar dan "achteruit" door het netwerk:

∂L/∂W = ∂L/∂y · ∂y/∂z₂ · ∂z₂/∂z₁ · ∂z₁/∂W

### Het algoritme

1. **Forward pass**: Bereken alle tussenresultaten en sla ze op ("cache")
2. **Backward pass**: Start bij ∂L/∂L = 1, werk achteruit door elke operatie
3. Bij elke operatie: vermenigvuldig de "upstream" gradiënt met de "lokale" gradiënt

upstream_gradient × local_gradient = downstream_gradient

In [ ]:
# Backprop voor ons simpele voorbeeld
# Stel de "target" is [0.8, 0.2] en we gebruiken MSE loss

y_target = np.array([0.8, 0.2])

# Loss = MSE = mean((y - y_target)²)
loss = np.mean((y - y_target) ** 2)
print(f"Loss = {loss:.6f}")
print()

# Backward pass
print("Backward Pass:")

# ∂L/∂y = 2(y - y_target) / n
dL_dy = 2 * (y - y_target) / len(y)
print(f"∂L/∂y = {dL_dy}")

# ∂L/∂z₂ = ∂L/∂y · ∂y/∂z₂ = ∂L/∂y · σ'(z₂)
dy_dz2 = sigmoid_derivative(z2)
dL_dz2 = dL_dy * dy_dz2
print(f"∂y/∂z₂ = σ'(z₂) = {dy_dz2}")
print(f"∂L/∂z₂ = {dL_dz2}")

# ∂L/∂b = ∂L/∂z₂ · ∂z₂/∂b = ∂L/∂z₂ · 1
dL_db = dL_dz2
print(f"∂L/∂b = {dL_db}")

# ∂L/∂z₁ = ∂L/∂z₂ · ∂z₂/∂z₁ = ∂L/∂z₂ · 1
dL_dz1 = dL_dz2
print(f"∂L/∂z₁ = {dL_dz1}")

# ∂L/∂W = ∂L/∂z₁ · ∂z₁/∂W = outer(∂L/∂z₁, x)
dL_dW = np.outer(dL_dz1, x)
print(f"∂L/∂W = \n{dL_dW}")

## 7.4 Lokale Gradiënten van Operaties

Elke operatie moet weten hoe hij zijn lokale gradiënt berekent. Dit is de afgeleide van de output naar de inputs.

### Basis operaties

**Optelling: z = x + y**
- ∂z/∂x = 1
- ∂z/∂y = 1

**Vermenigvuldiging: z = x · y**
- ∂z/∂x = y
- ∂z/∂y = x

**ReLU: z = max(0, x)**
- ∂z/∂x = 1 als x > 0, anders 0

**Sigmoid: z = σ(x)**
- ∂z/∂x = σ(x)(1 - σ(x))

**Matrix vermenigvuldiging: Z = X @ W**
- ∂L/∂X = ∂L/∂Z @ Wᵀ
- ∂L/∂W = Xᵀ @ ∂L/∂Z

In [ ]:
# Implementeer operaties met forward en backward

class ReLU:
    """ReLU activatie met forward en backward."""
    def forward(self, x):
        self.x = x
        return np.maximum(0, x)
    
    def backward(self, dout):
        return dout * (self.x > 0)

class Sigmoid:
    """Sigmoid activatie met forward en backward."""
    def forward(self, x):
        self.out = 1 / (1 + np.exp(-np.clip(x, -500, 500)))
        return self.out
    
    def backward(self, dout):
        return dout * self.out * (1 - self.out)

# Test ReLU
relu = ReLU()
x = np.array([-2, -1, 0, 1, 2])
y = relu.forward(x)
print("ReLU Forward:")
print(f"  x = {x}")
print(f"  y = {y}")

# Backward: stel upstream gradient is allemaal 1
dout = np.ones_like(y)
dx = relu.backward(dout)
print(f"ReLU Backward:")
print(f"  dout = {dout}")
print(f"  dx = {dx}")
print()
print("Merk op: gradiënt is 0 waar x ≤ 0 (geen leren mogelijk daar)")

## 7.5 Backprop Door een Mini-Netwerk

Laten we een volledig mini-netwerk uitwerken met concrete getallen. Dit helpt om de intuïtie te bouwen.

Netwerk: input → Linear → ReLU → Linear → MSE Loss

In [ ]:
# Mini-netwerk: 2 inputs → 3 hidden → 1 output

# Parameters
np.random.seed(42)
W1 = np.random.randn(3, 2) * 0.5  # 3x2: 2 inputs, 3 hidden
b1 = np.zeros(3)
W2 = np.random.randn(1, 3) * 0.5  # 1x3: 3 hidden, 1 output
b2 = np.zeros(1)

print("Netwerk structuur: 2 → 3 → 1")
print(f"W1 shape: {W1.shape}, b1 shape: {b1.shape}")
print(f"W2 shape: {W2.shape}, b2 shape: {b2.shape}")
print()
print("W1 =")
print(W1)
print(f"\nW2 = {W2}")

In [ ]:
# Forward pass
x = np.array([1.0, 2.0])  # Input
y_target = np.array([1.0])  # Target

print("=== FORWARD PASS ===")
print(f"Input: x = {x}")
print(f"Target: y_target = {y_target}")
print()

# Laag 1
z1 = W1 @ x + b1
a1 = np.maximum(0, z1)  # ReLU
print(f"z1 = W1 @ x + b1 = {z1}")
print(f"a1 = ReLU(z1) = {a1}")
print()

# Laag 2
z2 = W2 @ a1 + b2
y = z2  # Geen activatie op output (regressie)
print(f"z2 = W2 @ a1 + b2 = {z2}")
print(f"y (output) = {y}")
print()

# Loss
loss = 0.5 * np.sum((y - y_target) ** 2)  # 0.5 voor makkelijkere afgeleide
print(f"Loss = 0.5 * (y - y_target)² = {loss:.6f}")

In [ ]:
# Backward pass
print("=== BACKWARD PASS ===")
print()

# Start bij de loss
# ∂L/∂y = y - y_target
dL_dy = y - y_target
print(f"∂L/∂y = y - y_target = {dL_dy}")
print()

# Laag 2 backward
# z2 = W2 @ a1 + b2, y = z2
# ∂L/∂z2 = ∂L/∂y (want y = z2)
dL_dz2 = dL_dy
print(f"∂L/∂z2 = {dL_dz2}")

# ∂L/∂W2 = ∂L/∂z2 ⊗ a1 (outer product)
dL_dW2 = np.outer(dL_dz2, a1)
print(f"∂L/∂W2 = {dL_dW2}")

# ∂L/∂b2 = ∂L/∂z2
dL_db2 = dL_dz2
print(f"∂L/∂b2 = {dL_db2}")

# ∂L/∂a1 = W2ᵀ @ ∂L/∂z2
dL_da1 = W2.T @ dL_dz2
print(f"∂L/∂a1 = {dL_da1}")
print()

In [ ]:
# Laag 1 backward
# a1 = ReLU(z1)
# ∂L/∂z1 = ∂L/∂a1 * ReLU'(z1)
relu_grad = (z1 > 0).astype(float)  # 1 waar z1 > 0, anders 0
dL_dz1 = dL_da1 * relu_grad
print(f"ReLU'(z1) = {relu_grad}")
print(f"∂L/∂z1 = {dL_dz1}")

# ∂L/∂W1 = ∂L/∂z1 ⊗ x
dL_dW1 = np.outer(dL_dz1, x)
print(f"∂L/∂W1 = \n{dL_dW1}")

# ∂L/∂b1 = ∂L/∂z1
dL_db1 = dL_dz1
print(f"∂L/∂b1 = {dL_db1}")

In [ ]:
# Numerieke verificatie
def forward_network(x, W1, b1, W2, b2):
    z1 = W1 @ x + b1
    a1 = np.maximum(0, z1)
    z2 = W2 @ a1 + b2
    return z2

def compute_loss(x, y_target, W1, b1, W2, b2):
    y = forward_network(x, W1, b1, W2, b2)
    return 0.5 * np.sum((y - y_target) ** 2)

# Numerieke gradiënt voor W1[0,0]
h = 1e-5
W1_plus = W1.copy()
W1_plus[0, 0] += h
W1_minus = W1.copy()
W1_minus[0, 0] -= h

numerical_grad = (compute_loss(x, y_target, W1_plus, b1, W2, b2) - 
                  compute_loss(x, y_target, W1_minus, b1, W2, b2)) / (2 * h)

print("Verificatie:")
print(f"  ∂L/∂W1[0,0] analytisch: {dL_dW1[0,0]:.6f}")
print(f"  ∂L/∂W1[0,0] numeriek:   {numerical_grad:.6f}")
print(f"  Match: {np.isclose(dL_dW1[0,0], numerical_grad, rtol=1e-4)}")

## 7.6 Vectorized Backprop

In de praktijk werken we met batches van data. De wiskunde blijft hetzelfde, maar we moeten rekening houden met matrix dimensies.

Voor een batch van N samples:
- X heeft shape (N, input_dim)
- De gradiënten worden gesommeerd/gemiddeld over de batch

In [ ]:
# Volledige Layer class met forward en backward

class LinearLayer:
    """Lineaire laag: z = X @ W + b"""
    
    def __init__(self, input_dim, output_dim):
        # Xavier initialisatie
        self.W = np.random.randn(input_dim, output_dim) * np.sqrt(2.0 / input_dim)
        self.b = np.zeros(output_dim)
        
        # Gradiënten
        self.dW = None
        self.db = None
    
    def forward(self, X):
        self.X = X  # Cache voor backward
        return X @ self.W + self.b
    
    def backward(self, dout):
        # dout: gradiënt van loss naar output, shape (N, output_dim)
        N = self.X.shape[0]
        
        # Gradiënt naar parameters
        self.dW = self.X.T @ dout / N
        self.db = np.mean(dout, axis=0)
        
        # Gradiënt naar input (voor volgende laag)
        dX = dout @ self.W.T
        return dX

class ReLULayer:
    """ReLU activatie."""
    
    def forward(self, X):
        self.mask = (X > 0)
        return np.maximum(0, X)
    
    def backward(self, dout):
        return dout * self.mask

# Test
layer = LinearLayer(2, 3)
relu = ReLULayer()

X = np.array([[1.0, 2.0],
              [3.0, 4.0]])  # 2 samples

# Forward
z = layer.forward(X)
a = relu.forward(z)
print(f"Input X shape: {X.shape}")
print(f"Output a shape: {a.shape}")
print(f"Output:\n{a}")

## 7.7 De Volledige Training Loop

Nu bouwen we een volledig neuraal netwerk dat we kunnen trainen.

In [ ]:
class NeuralNetwork:
    """Een simpel neuraal netwerk met 1 hidden layer."""
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.layer1 = LinearLayer(input_dim, hidden_dim)
        self.relu = ReLULayer()
        self.layer2 = LinearLayer(hidden_dim, output_dim)
    
    def forward(self, X):
        """Forward pass."""
        self.z1 = self.layer1.forward(X)
        self.a1 = self.relu.forward(self.z1)
        self.z2 = self.layer2.forward(self.a1)
        return self.z2
    
    def backward(self, dout):
        """Backward pass."""
        dz2 = dout
        da1 = self.layer2.backward(dz2)
        dz1 = self.relu.backward(da1)
        dX = self.layer1.backward(dz1)
        return dX
    
    def update(self, learning_rate):
        """Update parameters met gradient descent."""
        self.layer1.W -= learning_rate * self.layer1.dW
        self.layer1.b -= learning_rate * self.layer1.db
        self.layer2.W -= learning_rate * self.layer2.dW
        self.layer2.b -= learning_rate * self.layer2.db

def mse_loss(y_pred, y_true):
    """Mean Squared Error loss."""
    return np.mean((y_pred - y_true) ** 2)

def mse_loss_gradient(y_pred, y_true):
    """Gradiënt van MSE loss."""
    return 2 * (y_pred - y_true) / y_true.shape[0]

In [ ]:
# Test op een eenvoudig regressieprobleem
np.random.seed(42)

# Genereer data: y = sin(x)
X_train = np.random.uniform(-3, 3, (200, 1))
y_train = np.sin(X_train)

# Netwerk
nn = NeuralNetwork(input_dim=1, hidden_dim=32, output_dim=1)

# Training
learning_rate = 0.1
n_epochs = 500
losses = []

for epoch in range(n_epochs):
    # Forward
    y_pred = nn.forward(X_train)
    
    # Loss
    loss = mse_loss(y_pred, y_train)
    losses.append(loss)
    
    # Backward
    dout = mse_loss_gradient(y_pred, y_train)
    nn.backward(dout)
    
    # Update
    nn.update(learning_rate)
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch:3d}: Loss = {loss:.6f}")

print(f"\nFinal loss: {losses[-1]:.6f}")

In [ ]:
# Visualiseer de fit
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Fit
X_test = np.linspace(-3, 3, 100).reshape(-1, 1)
y_test = nn.forward(X_test)

axes[0].scatter(X_train, y_train, alpha=0.3, label='Training data')
axes[0].plot(X_test, y_test, 'r-', linewidth=2, label='NN voorspelling')
axes[0].plot(X_test, np.sin(X_test), 'g--', linewidth=2, label='sin(x)')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('Neuraal netwerk leert sin(x)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss curve
axes[1].plot(losses)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MSE Loss')
axes[1].set_title('Training Loss')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7.8 Toepassing: MNIST Classificatie

Nu het grote moment: we trainen een neuraal netwerk op MNIST, de klassieke dataset van handgeschreven cijfers!

In [ ]:
# Laad MNIST data
from sklearn.datasets import fetch_openml

print("MNIST laden...")
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
X, y = mnist.data, mnist.target.astype(int)

# Normaliseer naar [0, 1]
X = X / 255.0

# Split in train/test
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Input dimensie: {X_train.shape[1]}")
print(f"Aantal klassen: {len(np.unique(y_train))}")

In [ ]:
# Bekijk wat voorbeelden
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(X_train[i].reshape(28, 28), cmap='gray')
    ax.set_title(f'Label: {y_train[i]}')
    ax.axis('off')
plt.suptitle('MNIST voorbeelden', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Softmax en Cross-Entropy voor classificatie

def softmax(x):
    """Softmax functie (numeriek stabiel)."""
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def cross_entropy_loss(y_pred, y_true):
    """Cross-entropy loss voor classificatie."""
    N = y_pred.shape[0]
    # One-hot encoding
    y_one_hot = np.zeros_like(y_pred)
    y_one_hot[np.arange(N), y_true] = 1
    # Cross-entropy
    loss = -np.sum(y_one_hot * np.log(y_pred + 1e-10)) / N
    return loss

def cross_entropy_gradient(y_pred, y_true):
    """Gradiënt van softmax + cross-entropy."""
    N = y_pred.shape[0]
    grad = y_pred.copy()
    grad[np.arange(N), y_true] -= 1
    return grad / N

In [ ]:
# MNIST classifier

class MNISTClassifier:
    """Neuraal netwerk voor MNIST classificatie."""
    
    def __init__(self, input_dim=784, hidden_dim=128, output_dim=10):
        self.layer1 = LinearLayer(input_dim, hidden_dim)
        self.relu = ReLULayer()
        self.layer2 = LinearLayer(hidden_dim, output_dim)
    
    def forward(self, X):
        self.z1 = self.layer1.forward(X)
        self.a1 = self.relu.forward(self.z1)
        self.z2 = self.layer2.forward(self.a1)
        self.probs = softmax(self.z2)
        return self.probs
    
    def backward(self, y_true):
        dout = cross_entropy_gradient(self.probs, y_true)
        da1 = self.layer2.backward(dout)
        dz1 = self.relu.backward(da1)
        self.layer1.backward(dz1)
    
    def update(self, learning_rate):
        self.layer1.W -= learning_rate * self.layer1.dW
        self.layer1.b -= learning_rate * self.layer1.db
        self.layer2.W -= learning_rate * self.layer2.dW
        self.layer2.b -= learning_rate * self.layer2.db
    
    def predict(self, X):
        probs = self.forward(X)
        return np.argmax(probs, axis=1)
    
    def accuracy(self, X, y):
        preds = self.predict(X)
        return np.mean(preds == y)

In [ ]:
# Train het netwerk!
np.random.seed(42)

model = MNISTClassifier(hidden_dim=128)

# Hyperparameters
learning_rate = 0.5
batch_size = 128
n_epochs = 10

# Training geschiedenis
train_losses = []
train_accs = []
test_accs = []

n_batches = len(X_train) // batch_size

print("Training starten...")
print(f"Batch size: {batch_size}, Batches per epoch: {n_batches}")
print()

for epoch in range(n_epochs):
    # Shuffle data
    indices = np.random.permutation(len(X_train))
    X_shuffled = X_train[indices]
    y_shuffled = y_train[indices]
    
    epoch_loss = 0
    
    for batch in range(n_batches):
        start = batch * batch_size
        end = start + batch_size
        
        X_batch = X_shuffled[start:end]
        y_batch = y_shuffled[start:end]
        
        # Forward
        probs = model.forward(X_batch)
        loss = cross_entropy_loss(probs, y_batch)
        epoch_loss += loss
        
        # Backward
        model.backward(y_batch)
        
        # Update
        model.update(learning_rate)
    
    # Evalueer
    avg_loss = epoch_loss / n_batches
    train_acc = model.accuracy(X_train[:5000], y_train[:5000])  # Subset voor snelheid
    test_acc = model.accuracy(X_test, y_test)
    
    train_losses.append(avg_loss)
    train_accs.append(train_acc)
    test_accs.append(test_acc)
    
    print(f"Epoch {epoch+1:2d}: Loss = {avg_loss:.4f}, Train acc = {train_acc:.4f}, Test acc = {test_acc:.4f}")

print(f"\nFinale test accuracy: {test_accs[-1]*100:.2f}%")

In [ ]:
# Visualiseer training
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(train_losses, 'b-', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

axes[1].plot(train_accs, 'b-', linewidth=2, label='Train')
axes[1].plot(test_accs, 'r-', linewidth=2, label='Test')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Bekijk voorspellingen
fig, axes = plt.subplots(2, 5, figsize=(14, 6))

# Random test samples
indices = np.random.choice(len(X_test), 10, replace=False)

for i, (ax, idx) in enumerate(zip(axes.flatten(), indices)):
    img = X_test[idx].reshape(28, 28)
    true_label = y_test[idx]
    pred_label = model.predict(X_test[idx:idx+1])[0]
    
    ax.imshow(img, cmap='gray')
    color = 'green' if pred_label == true_label else 'red'
    ax.set_title(f'Pred: {pred_label}, True: {true_label}', color=color)
    ax.axis('off')

plt.suptitle('Model voorspellingen (groen=correct, rood=fout)', fontsize=14)
plt.tight_layout()
plt.show()

## 7.9 Samenvatting

### Wat hebben we bereikt?

We hebben een neuraal netwerk **from scratch** gebouwd en getraind! Geen TensorFlow, geen PyTorch - alleen NumPy en ons begrip van de wiskunde.

### Kernconcepten

**Backpropagation** is de efficiënte toepassing van de kettingregel door een computational graph. De forward pass berekent de output en slaat tussenresultaten op. De backward pass berekent gradiënten van de loss naar alle parameters.

Elke operatie heeft een **lokale gradiënt** die bepaalt hoe de output verandert met de input. De **kettingregel** combineert deze lokale gradiënten tot de totale gradiënt.

### De complete training loop

1. **Forward pass**: bereken output en loss
2. **Backward pass**: bereken gradiënten
3. **Update**: pas parameters aan met gradient descent
4. Herhaal voor elke batch, voor meerdere epochs

### Einde Deel 2: Calculus

We hebben nu alle tools om neurale netwerken te begrijpen en te trainen:
- **Lineaire algebra** (Deel 1): data flow door het netwerk
- **Calculus** (Deel 2): leren via gradient descent en backpropagation

In Deel 3 (Statistiek en Kansrekening) zullen we dieper ingaan op loss functies, output interpretaties en model evaluatie.

---

**Mathematical Foundations** | Les 7 van 12 | IT & Artificial Intelligence

---